In [9]:
import random
from math import gcd

# Перевіряємо на простоту числа n з використанням тесту Міллера-Рабіна
def miller_rabin(n, k=40):
    if n <= 1 or (n > 3 and n % 2 == 0):
      return False
    if n <= 3:
      return True

    # Представляємо n-1 у вигляді (2^r)*d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Тестування
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Генеруємо випадкове просте число
def generating_prime(bit_length):
    while True:
        number = random.getrandbits(bit_length) | (1 << (bit_length - 1)) | 1
        if miller_rabin(number):
            return number

prime = generating_prime(256)
print("Просте число:", prime)

Просте число: 100574064494694283857249577410038087635246173863423551143772504817695638085723


In [10]:
# Генеруємо дві пари простих чисел p, q і p1, q1
def generating_pairs(bit_length):
    while True:
        p = generating_prime(bit_length)
        q = generating_prime(bit_length)
        p1 = generating_prime(bit_length)
        q1 = generating_prime(bit_length)
        if p*q <= p1*q1:
            return (p, q), (p1, q1)

bit_length = 256
(pair1, pair2) = generating_pairs(bit_length)
(p, q) = pair1
(p1, q1) = pair2

print("Пара для A: p =", p, ", q =", q)
print("Пара для B: p1 =", p1, ", q1 =", q1)


Пара для A: p = 75696055949374935531539360269919943641097864649664761825099781654484847217323 , q = 81059111580080410830968372851627995147020976835150522596362127618916968815207
Пара для B: p1 = 81734248121107113957517993519244515354238157209037091213785191711352798812171 , q1 = 83622390664251948001183617437572689071602655261494339700910807651229859167739


In [11]:
def inverse(e, phi):
    # Обчислюємо e (mod phi) за розширеним алгоритмом Евкліда
    a, b = e, phi
    x0, x1 = 1, 0
    while b != 0:
        q, a, b = a // b, b, a % b
        x0, x1 = x1, x0 - q*x1
    return x0 % phi

# Генеруємо пару ключів RSA: (n, e) та (d, p, q)
def generating_rsa_keys(p, q):
    n = p*q
    phi = (p - 1)*(q - 1)
    e = 65537
    d = inverse(e, phi)
    return (n, e), (d, p, q)

# Генеруємо ключі для A і В
pub_a, priv_a = generating_rsa_keys(p, q)
pub_b, priv_b = generating_rsa_keys(p1, q1)

print("Відкритий ключ A:", pub_a)
print("Секретний ключ A:", priv_a)
print("Відкритий ключ B:", pub_b)
print("Секретний ключ B:", priv_b)

Відкритий ключ A: (6135855045372392513266273295385235674181490342646385302088731973962844183876410880579820340514288427465113398354026689281878120608678470917417082156230861, 65537)
Секретний ключ A: (5604724452388313554391750988829848000354009763525690979511418754128380979043411766844399065408741890212184874700670488134087688327682402112753303933436577, 75696055949374935531539360269919943641097864649664761825099781654484847217323, 81059111580080410830968372851627995147020976835150522596362127618916968815207)
Відкритий ключ B: (6834813227032119847987495530346088987022118425012939115733442979599822628804100552377257207519840601240452057252169771257783974697783174173094933843751369, 65537)
Секретний ключ B: (1147913227488938205392348799952994514246188527154392493505622882897527315489645768031697508826707143586399348569594541135718221310685892537929291690217533, 81734248121107113957517993519244515354238157209037091213785191711352798812171, 8362239066425194800118361743757268907160265526149

In [16]:
# Шифруємо повідомлення
def encrypting_message(message, pub_key):
    n, e = pub_key
    return pow(message, e, n)

# Розшифровуємо повідомлення
def decrypting_message(cipher, priv_key):
    d, p, q = priv_key
    n = p*q
    return pow(cipher, d, n)

# Підписуємо повідомлення
def signing(message, priv_key):
    d, p, q = priv_key
    n = p*q
    return pow(message, d, n)

# Перевіряємо підпис
def verifying(signature, pub_key, message):
    n, e = pub_key
    return pow(signature, e, n) == message

message = random.randint(1, pub_a[0] - 1)
cipher = encrypting_message(message, pub_a)
decrypted_message = decrypting_message(cipher, priv_a)
signature = signing(message, priv_a)
is_valid = verifying(signature, pub_a, message)

print("Повідомлення:", message)
print("Шифротекст:", cipher)
print("Розшифроване повідомлення:", decrypted_message)
print("Цифровий підпис:", signature)
print("Перевірка підпису:", is_valid)


Повідомлення: 783624678796015798344049835263908947130749282836829043792062260763326628825902518355803798808899248961012867517979048816795708134357560437932351557003825
Шифротекст: 3327221206454051302028899961144073105520745337996975207329412543186601821945636019426313885537880975464472763404541605145542987152964164726647331961186043
Розшифроване повідомлення: 783624678796015798344049835263908947130749282836829043792062260763326628825902518355803798808899248961012867517979048816795708134357560437932351557003825
Цифровий підпис: 197197097930744187988176282083390673794099045631325477458004969985239971018584251744350567357448428958983163094518216349706567783634191915372766406213250
Перевірка підпису: True


In [17]:
# Відправляємо ключ k з підтвердженням справжності
def send_key(secret_key, recipient_pub_key, k):
    signature = signing(k, secret_key)  # Підписуємо ключ
    encrypted_key = encrypting_message(k, recipient_pub_key)  # Шифруємо ключ
    encrypted_signature = encrypting_message(signature, recipient_pub_key)  # Шифруємо підпис
    return encrypted_key, encrypted_signature

# Отримуємо ключ k, розшифровуємо і перевіряємо його підпис
def receive_key(encrypted_key, encrypted_signature, recipient_priv_key, sender_pub_key):
    decrypted_key = decrypting_message(encrypted_key, recipient_priv_key)  # Розшифровуємо ключ
    decrypted_signature = decrypting_message(encrypted_signature, recipient_priv_key)  # Розшифровуємо підпис
    # Перевіряємо справжність підпису
    if verifying(decrypted_signature, sender_pub_key, decrypted_key):
        return decrypted_key
    else:
        print("Невірний підпис! Ключ не є автентичним.")

k = random.randint(1, pub_b[0] - 1)
# Відправник A відправляє ключ B
encrypted_key, encrypted_signature = send_key(priv_a, pub_b, k)

# Отримувач B отримує ключ від A
try:
    received_key = receive_key(encrypted_key, encrypted_signature, priv_b, pub_a)
    print("Переданий ключ:", k)
    print("Отриманий ключ:", received_key)
    print("Перевірка пройшла успішно!" if received_key == k else "Ключі не співпадають!")
except ValueError as e:
    print("Помилка перевірки:", str(e))


Переданий ключ: 5930522795810190933654409450146250309212382864651123148892872366586279567005733229459805595364552179278834167855187233976049877736017693865941145418571698
Отриманий ключ: 5930522795810190933654409450146250309212382864651123148892872366586279567005733229459805595364552179278834167855187233976049877736017693865941145418571698
Перевірка пройшла успішно!
